# Hate Speech Detection on the Davidson Dataset
## Models : MLP + SVM + XGB - TF-IDF Embedding

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
import pandas as pd
import numpy as np
from itertools import chain
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import Embedding, BatchNormalization, Dense, Flatten, Dropout, Bidirectional,Flatten, GlobalMaxPool1D,Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau,CSVLogger
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras.utils import np_utils
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import xgboost as xgb
from sklearn import svm

/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'tensorflow'

## Data Extraction

In [ ]:
#Data Reading for Davidson Dataset
nRowsRead = None # specify 'None' to read complete file
df0 = pd.read_csv('../input/hateoffensive/labeled_data.csv', delimiter=',', nrows = nRowsRead)
nRow, nCol = df0.shape
print('There are {} rows and {} columns'.format(nRow, nCol))

#Doing Transformation
c=df0['class']
df0.rename(columns={'tweet' : 'text',
                   'class' : 'category'}, 
                    inplace=True)
a=df0['text']
b=df0['category'].map({0: 'hate_speech', 1: 'offensive_language',2: 'neither'})
df= pd.concat([a,b,c], axis=1)
df.rename(columns={'class' : 'label'}, 
                    inplace=True)

hate, ofensive, neither = np.bincount(df['label'])
total = hate + ofensive + neither
print('Examples:\n    Total: {}\n    hate: {} ({:.2f}% of total)\n'.format(
    total, hate, 100 * hate / total))
print('Examples:\n    Total: {}\n    Ofensive: {} ({:.2f}% of total)\n'.format(
    total, ofensive, 100 * ofensive / total))
print('Examples:\n    Total: {}\n    Neither: {} ({:.2f}% of total)\n'.format(
    total, neither, 100 * neither / total))


In [ ]:
x= df['text']
y=df['label']

hate, ofensive, neither = np.bincount(df['label'])
total = hate + ofensive + neither
print('Examples:\n    Total: {}\n    hate: {} ({:.2f}% of total)\n'.format(
    total, hate, 100 * hate / total))
print('Examples:\n    Total: {}\n    Ofensive: {} ({:.2f}% of total)\n'.format(
    total, ofensive, 100 * ofensive / total))
print('Examples:\n    Total: {}\n    Neither: {} ({:.2f}% of total)\n'.format(
    total, neither, 100 * neither / total))

In [ ]:
#defining_class_weight for each class
weight_class1 = (1 / hate)*(total)/3.0 
weight_class2 = (1 / ofensive)*(total)/3.0
weight_class3 = (1 / neither)*(total)/3.0
class_weight = {0: weight_class1, 1: weight_class2, 2: weight_class3}

tfidf = TfidfVectorizer(binary=True)

In [ ]:
#splitting the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

#for SVM and XGB
Tfidf_vect = TfidfVectorizer(use_idf=True)
Tfidf_vect.fit(x)
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)


#fitting and transforming datasets
x = tfidf.fit_transform(x_train).todense()
x_test = tfidf.transform(x_test).todense()    

y_train_cat = np_utils.to_categorical(y_train)

y_test_= y_test
y_test_cat = np_utils.to_categorical(y_test)

In [ ]:
#creating model
def MLP():
  model = Sequential()
  model.add(Dense(512, activation='relu')) #dense_layer
  model.add(Dropout(0.2))#dropout_layer
  model.add(Dense(3, activation="softmax", name='classifier')) #classification_layer
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
#building model
mlp_model=MLP()

In [ ]:
Train_X_Tfidf.sort_indices()
Test_X_Tfidf.sort_indices()

In [ ]:
#running model
mlp_history = mlp_model.fit(Train_X_Tfidf, 
                      y_train_cat, 
                      batch_size=128,
                      epochs=20, 
                      validation_data=(Test_X_Tfidf, y_test_cat))

In [ ]:
#creating graphs for accuracy and loss
#fining_values
history_dict = mlp_history.history
print(history_dict.keys())
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = range(1, len(acc) + 1)


In [ ]:
#defining_figures
fig = plt.figure(figsize=(12, 10))
fig.tight_layout()
#1st_figure
plt.subplot(2, 1, 1)
plt.plot(epochs, loss, 'r', label='Training-loss')#training_loss
plt.plot(epochs, val_loss, 'b', label='Validation-loss')#validation_loss
plt.title('Training & validation Loss')
plt.xlabel('Epochs-Count')
plt.ylabel('Loss')
plt.legend()
#2nd_Figure
plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training-accuracy')#training_accuracy
plt.plot(epochs, val_acc, 'b', label='Validation-accuracy')#validation_accuracy
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs-Count')
plt.ylabel('Acc')
plt.legend(loc='lower right')

# y_test = np.array(list(chain.from_iterable(y_test)))

In [ ]:
#predicting
y_pred = mlp_model.predict(Test_X_Tfidf, verbose=0)

In [ ]:
#creating reports
print(confusion_matrix(y_test_, y_pred.argmax(axis = -1)))
print(classification_report(y_test_, y_pred.argmax(axis = -1)))

## SVM Model

In [ ]:
#running SVM
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',verbose=True,class_weight=class_weight, decision_function_shape="ovo")
SVM.fit(Train_X_Tfidf,y_train)

In [ ]:
#predciting
predictions_SVM = SVM.predict(Test_X_Tfidf)
print(classification_report(y_test,predictions_SVM))

## XGB Model

In [ ]:
#running XGB
xgb_classifier = xgb.XGBClassifier()
xgb= xgb_classifier.fit(Train_X_Tfidf,y_train)

In [ ]:
#predciting
predictions_xgb = xgb.predict(Test_X_Tfidf)
print(classification_report(y_test,predictions_xgb))


In [ ]:
# xgb.save_model("xgb_classifier.json")

In [ ]:
# import xgboost as xg
# booster = xg.Booster()
# booster.load_model("../input/hsd-xgb-classifier/xgb_classifier.json")


In [ ]:
# import json
# from collections import OrderedDict
# tf_idf = {}
# for index,val in enumerate(Tfidf_vect.get_feature_names_out()):
#     tf_idf[val] ={"idf":Tfidf_vect.idf_[index],"index":index}
# with open('idf_values.json', 'w') as f:
#     json.dump(tf_idf, f)